In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import time
import torch.optim as optim
import os
import sys

from torchvision import transforms
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader

In [ ]:
PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/MLME2025_project'
CITYSCAPES_DIR = '/content/drive/MyDrive/Cityspaces/'
GTA5_DIR = '/content/drive/MyDrive/GTA5/'
BEST_MODEL_SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/MLME2025_project/models/BiSeNet/checkpoints_training_DOMAINSHIFT/best_model_BiSeNet_DOMAINSHIFT.pth'
LAST_EPOCH_SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/MLME2025_project/models/BiSeNet/checkpoints_training_DOMAINSHIFT/last_epoch_BiSeNet_DOMAINSHIFT.pth'


H_CITYSCAPES = 512
W_CITYSCAPES = 1024

H_GTA5 = 720
W_GTA5 = 1280

NUM_CLASSES = 19
BATCH_SIZE = 4
NUM_WORKERS = 4
LEARNING_RATE = 0.025
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0001

In [ ]:
os.chdir(PROJECT_PATH)
sys.path.append(os.getcwd())

from datasets.cityscapes import CityScapes
from datasets.gta5 import GTA5
from utils.utils import poly_lr_scheduler_with_backbone, fast_hist, per_class_iou, mean_iou
from models.BiSeNet.build_bisenet import BiSeNet

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# transformers data

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((H_GTA5, W_GTA5), interpolation=InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((H_CITYSCAPES, W_CITYSCAPES), interpolation=InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# transformers label

label_transforms = {
    'train': transforms.Compose([
        transforms.Resize((H_GTA5, W_GTA5), interpolation=InterpolationMode.NEAREST),
        transforms.Lambda(lambda x: torch.from_numpy(np.array(x)).long())
    ]),
    'val': transforms.Compose([
        transforms.Resize((H_CITYSCAPES, W_CITYSCAPES), interpolation=InterpolationMode.NEAREST),
        transforms.Lambda(lambda x: torch.from_numpy(np.array(x)).long())
    ])
}


# datsets

GTA_to_Cityscapes_datasets = {
    'train': GTA5(
        data_path=GTA5_DIR,
        transform=data_transforms['train'],
        label_transform=label_transforms['train']
    ),
    'val': CityScapes(
        data_path=CITYSCAPES_DIR,
        split='val',
        transform=data_transforms['val'],
        label_transform=label_transforms['val']
    )

}

# dataloaders

GTA_to_Cityscapes_dataloaders = {
    x: torch.utils.data.DataLoader(
        GTA_to_Cityscapes_datasets[x],
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=True,
        pin_memory=True
    )
    for x in ['train', 'val']
}

# dataset sizes

dataset_sizes = {x: len(GTA_to_Cityscapes_datasets[x]) for x in ['train', 'val']}

In [ ]:
def train_model(model, data_loader, dataset_sizes, criterion, optimizer,
                last_epoch_save_path, best_model_save_path, num_epochs=1,
                init_lr=0.01, prev_num_epoch=0, prev_best_miou=0,
                total_number_epochs=50):

    since = time.time()

    best_miou = prev_best_miou
    best_per_class_iou = None

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        since_epoch = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0

            hist = np.zeros((NUM_CLASSES, NUM_CLASSES))
            miou = 0

            for batch in data_loader[phase]:
                inputs = batch['x']
                labels = batch['y']

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    if phase == 'train':
                        loss = criterion(outputs[0], labels)
                        running_loss += loss.item()
                        loss.backward()

                        optimizer.step()

                        preds = torch.argmax(outputs[0], dim=1)
                        hist += fast_hist(
                                 preds.cpu().data.numpy().flatten(),
                                 labels.cpu().data.numpy().flatten(),
                                 NUM_CLASSES
                                )

                    else:
                        loss = criterion(outputs, labels)
                        running_loss += loss.item()

                        preds = torch.argmax(outputs, dim=1)
                        hist += fast_hist(
                                 preds.cpu().data.numpy().flatten(),
                                 labels.cpu().data.numpy().flatten(),
                                 NUM_CLASSES
                                )

            ious = per_class_iou(hist) * 100
            miou = mean_iou(ious)

            epoch_loss = running_loss / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {miou:.4f}')

        # update learning rate with poly_lr_scheduler
        next_lr = poly_lr_scheduler_with_backbone(optimizer, init_lr, prev_num_epoch, total_number_epochs)
        prev_num_epoch += 1

        # save the best model
        if miou > best_miou:
            best_miou = miou
            best_per_class_iou = ious
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, best_model_save_path)

        time_epoch = time.time() - since_epoch
        print(f'Epoch complete in {time_epoch // 60:.0f}m {time_epoch % 60:.0f}s')
        print(f'Next Learning Rate: {next_lr}')
        print()

        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, last_epoch_save_path)

    time_elapsed = time.time() - since
    print('-' * 20)
    print()
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val MIOU: {best_miou:4f}')
    print(f'Best val per class IOU: {best_per_class_iou}')
    print()
    print(f'Total Epochs completed: {prev_num_epoch}')

    return model, time_elapsed, best_miou, best_per_class_iou

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
context_path = 'resnet18'

model = BiSeNet(num_classes=NUM_CLASSES, context_path=context_path)
model = model.to(device)

optimizer = optim.SGD(
    params=[
    {'params': model.context_path.parameters(), 'lr': LEARNING_RATE * 0.1, 'initial_lr': LEARNING_RATE * 0.1},
    {'params': [p for module in model.mul_lr for p in module.parameters()], 'lr': LEARNING_RATE, 'initial_lr': LEARNING_RATE}
    ],
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

criterion = nn.CrossEntropyLoss(ignore_index=255)

# load previous checkpoint
checkpoint = torch.load(LAST_EPOCH_SAVE_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 231MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 247MB/s]


"\ncheckpoint = torch.load(LAST_EPOCH_SAVE_PATH)\nmodel.load_state_dict(checkpoint['model_state_dict'])\noptimizer.load_state_dict(checkpoint['optimizer_state_dict'])"

In [ ]:
model, time_elapsed, best_miou, best_per_class_iou = train_model(
    model=model,
    data_loader=GTA_to_Cityscapes_dataloaders,
    dataset_sizes=dataset_sizes,
    criterion=criterion,
    optimizer=optimizer,
    last_epoch_save_path=LAST_EPOCH_SAVE_PATH,
    best_model_save_path=BEST_MODEL_SAVE_PATH,
    num_epochs=50,
    init_lr=LEARNING_RATE,
    prev_num_epoch=0,
    prev_best_miou=0,
    total_number_epochs=50
)

Epoch 1/50
----------
train Loss: 0.1097 Acc: 31.0508
val Loss: 0.6721 Acc: 14.4778
Epoch complete in 24m 2s
Next Learning Rate: [0.0025000000000000005, 0.025]

Epoch 2/50
----------
train Loss: 0.0714 Acc: 42.0193
val Loss: 0.4910 Acc: 15.9249
Epoch complete in 14m 7s
Next Learning Rate: [0.0024549546664914385, 0.024549546664914382]

Epoch 3/50
----------
train Loss: 0.0607 Acc: 49.2845
val Loss: 0.6036 Acc: 18.5184
Epoch complete in 14m 25s
Next Learning Rate: [0.002409817303146378, 0.024098173031463777]

Epoch 4/50
----------
train Loss: 0.0541 Acc: 55.0852
val Loss: 0.7014 Acc: 17.0376
Epoch complete in 13m 60s
Next Learning Rate: [0.0023645857984465808, 0.023645857984465803]

Epoch 5/50
----------
train Loss: 0.0505 Acc: 57.7079
val Loss: 0.6992 Acc: 19.3014
Epoch complete in 13m 46s
Next Learning Rate: [0.002319257946408321, 0.023192579464083207]

Epoch 6/50
----------
train Loss: 0.0472 Acc: 59.7249
val Loss: 0.8464 Acc: 18.5444
Epoch complete in 14m 2s
Next Learning Rate: [0.00